In [1]:
from SyMBac.phase_contrast_drawing import run_simulation, get_trench_segments, draw_scene
from SyMBac.general_drawing import generate_curve_props, gen_cell_props_for_draw, get_space_size, convolve_rescale

In [2]:
resize_amount = 3
pix_mic_conv = 0.0655 ##0.108379937 micron/pix for 60x, 0.0655 for 100x
scale = pix_mic_conv / resize_amount 
sim_length = 1000

In [3]:
cell_timeseries, space = run_simulation(
    trench_length=15, 
    trench_width=1.5, 
    cell_max_length=1.65, #6, long cells # 1.65 short cells
    cell_width= 1, #1 long cells # 0.95 short cells
    sim_length = sim_length,
    pix_mic_conv = pix_mic_conv,
    gravity=0,
    phys_iters=20,
    max_length_var = 0.1,
    width_var = 0.3,
    save_dir="/tmp/"
) # growth phase

In [4]:
main_segments = get_trench_segments(space)
ID_props = generate_curve_props(cell_timeseries)

In [5]:
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

cell_timeseries_properties = Parallel(n_jobs=-1)(
    delayed(gen_cell_props_for_draw)(a, ID_props) for a in tqdm(cell_timeseries, desc='Timeseries Properties'))

do_transformation = True
offset = 30
label_masks = True
space_size = get_space_size(cell_timeseries_properties)
scenes = Parallel(n_jobs=-1)(delayed(draw_scene)(
    cell_properties, do_transformation, space_size, offset, label_masks) for cell_properties in tqdm(cell_timeseries_properties, desc='Scene Draw:'))

Timeseries Properties:   0%|          | 0/998 [00:00<?, ?it/s]

Scene Draw::   0%|          | 0/998 [00:00<?, ?it/s]

In [6]:
import pickle
cell_timeseries_properties_file = open('cell_timeseries_properties_small.p', 'wb')
pickle.dump(cell_timeseries_properties, cell_timeseries_properties_file)
cell_timeseries_properties_file.close()

main_segments_file = open('main_segments_small.p', 'wb')
pickle.dump(main_segments, main_segments_file)
main_segments_file.close()